<div style='width: auto; float: center'/>
<table align="center">
    <tr>
        <th>
                <img src="STATIC/logoUnal.png" style='height: 150px; width: auto; object-fit: contain;float: right'/>                                		
        </th>
        <th>
                <img src="STATIC/EMC_ACI.svg" style='height: 150px; width: auto; object-fit: contain;float: right'/>		
        </th>        
    </tr>
</table>

</div>

<div >
  <h1 align="center">    Introducción a la Gestión de Activos -2028644    </h1>  
			<h2 align="center">    David L. Alvarez A, Ph.D    </h2>
			<h3 align="center">    dlalvareza@unal.edu.co      </h3>
            <h4 align="center">    Copyright (c) 2020 dlalvareza    </h4>
</div>	

# IEEE 39-Bus System
![title](STATIC/AUX_EXAMPLE.svg)

## Cost of replacing assets
| Asset Type| Cost |
| --- | --- | 
| TR  | 2e4 | 
| OHL | 2.25e4 |
| CB  | 5e3|
| AUX | 2e3 |

### Neccesary libraries

In [1]:
import sys
#!pip install --upgrade XlsxWriter
#!pip install --upgrade openpyxl
#!pip install --upgrade pandas
#!pip install --upgrade inspyred
#!pip install --upgrade geopandas
#!pip install --upgrade contextily
#!pip install --upgrade tables
#!pip install --upgrade seaborn
#!pip install --upgrade bokeh 
#!pip install selenium
#!conda install -c conda-forge firefox geckodriver

### Add PywerAM paths  

In [2]:
sys.path.insert(0,'CASES/02_Wakanda/')
sys.path.insert(0,'APM/BIN/')
import datetime 
import pandas as pd
#import numpy as np

### Assessment parameter

In [3]:
date_beg  = datetime.date.today()
#N_years   = 25
N_years   = 25
d_day_for = int(N_years*365.25 )

# 2.  Condition Estimation

### Import run condition from APM Library

In [4]:
from APM_Run import run_condition
from PywerAM_bokeh_tools import plot_condition_forecast, Plot_HI_Forecast_Stacked,plot_condition_assessment,plot_decision_making, plot_scenario

## 2.1 Run asset fleet condition
![title](STATIC/07_Health_Index_Computation.svg)

In [5]:
_,_,assets = run_condition()

APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: invalid value encountered in power
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: divide by zero encountered in power
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module.py:222: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  dic =  df.to_dict('r')
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: overflow encountered in power
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: overflow encountered in double_scalars
  return (1 - np.exp(-((t*beta)**(1/alpha))))


In [6]:
asset_list_id   = list(assets.Asset_Portfolio.keys())        # List of asset id
asset_list_name = list(assets.Asset_Portfolio_List.Name)
asset_id        = asset_list_id[0]
Cond_list       = assets.Asset_Portfolio[asset_id].cond.keys()
l_asset         = assets.Asset_Portfolio[asset_id]

\begin{equation*}
S \left( t \right) = 1- e ^{-\left(\frac{t}{l}\right)^m}
\end{equation*}

In [7]:
from ipywidgets import interact
from ipywidgets import fixed
from bokeh.io import export_svgs

from bokeh.io import push_notebook, show, output_notebook
output_notebook()

def update_aset_cond(Asset_Name):
    df             = assets.Asset_Portfolio_List
    asset_id = df[df['Name'] == Asset_Name].index.values[0]
    l_asset  = assets.Asset_Portfolio[asset_id]
    Cond_list = list(l_asset.cond.keys())
    interact(update_condition_plot,Cond=Cond_list,asset=fixed(l_asset))
    
def update_condition_plot(Cond,asset):
    p  = plot_condition_forecast(Cond,asset,d_day_for)
    show(p, notebook_handle=True)
    push_notebook()
    
interact(update_aset_cond, Asset_Name=asset_list_name)

Loading BokehJS ...

interactive(children=(Dropdown(description='Asset_Name', options=('TR_1', 'TR_2', 'TR_3', 'TR_4', 'TR_5', 'TR_…

<function __main__.update_aset_cond(Asset_Name)>

In [8]:
df        = assets.Asset_Portfolio[asset_id].cond['TDCG'].historic_data
cond_pred = assets.Asset_Portfolio[asset_id].cond['TDCG'].eval_cond_fit_func(date_beg)
print(cond_pred)

0.7914945201101267


## 2.2 Health index forecast

In [9]:
%%time
date_for      = [date_beg + datetime.timedelta(days=x) for x in range(d_day_for)]
hi            = [assets.Asset_Portfolio[asset_id].Eval_Asset_Condition(date) for date in date_for]

CPU times: user 231 ms, sys: 0 ns, total: 231 ms
Wall time: 230 ms


\begin{equation*}
 HI \left( {\color{DColor9}t} \right) = \cfrac{\sum\limits_{n=1}^{N}\beta_n(S\left(  {\color{DColor9}t}\right)_n \cdot w_n)}{\sum\limits_{n=1}^N\beta_n w_n}
\end{equation*}

In [10]:
def update_HI_plot(Asset_Name):
    df             = assets.Asset_Portfolio_List
    asset_id       = df[df['Name'] == Asset_Name].index.values[0]      #global asset_id
    print('Asset Id:' +str(asset_id))
    asset         = assets.Asset_Portfolio[asset_id]
    date_for      = [date_beg + datetime.timedelta(days=x) for x in range(d_day_for)]
    
    fig           = Plot_HI_Forecast_Stacked(asset,date_for)
    show(fig, notebook_handle=True)
    push_notebook()

    
interact(update_HI_plot, Asset_Name=asset_list_name)

interactive(children=(Dropdown(description='Asset_Name', options=('TR_1', 'TR_2', 'TR_3', 'TR_4', 'TR_5', 'TR_…

<function __main__.update_HI_plot(Asset_Name)>

## 2.3 Load criticality data

In [11]:
from ARM_Run import load_criticality
df_ACP           = load_criticality()    # Montecarlo simulations
df_ACP

RESULTS/Results_ACM.h5


,TR_1,TR_2,TR_3,TR_4,TR_5,TR_6,TR_7,TR_8,TR_9,TR_10,...,LN_01_39,LN_09_39,LN_05_08,LN_05_06,LN_06_07,LN_21_22,Date,Cr,SAIDI,Ite
0,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,2020-12-18 06:00:00,0.0,0.0,0
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,2020-12-18 07:00:00,0.0,0.0,0
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,2020-12-18 08:00:00,0.0,0.0,0
3,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,2020-12-18 09:00:00,0.0,0.0,0
4,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,2020-12-18 10:00:00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078683,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2045-11-25 00:00:00,0.0,0.0,999
3078684,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2045-11-25 01:00:00,0.0,0.0,999
3078685,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2045-11-25 02:00:00,0.0,0.0,999
3078686,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2045-11-25 03:00:00,0.0,0.0,999


### Probabilty of failure

In [12]:
# Load fixed criticality
df_Fixed_Cr           = load_criticality(cr_type='CASES/02_Wakanda/01_Data/CRITICALITY.xlsx',assets=assets.Asset_Portfolio_List)
df_Fixed_Cr

     Name Type  MTTR Disc_Type Asset_To_Disconet  Incomes  CAPEX  OPEX  Cr_Fin
ID                                                                            
1    TR_1   TR     8        TR              TR_1     7000  63000  5250   84000
2    TR_2   TR     8        TR              TR_2    10000  90000  7500  120000
3    TR_3   TR     8        TR              TR_3    10000  90000  7500  120000
4    TR_4   TR     8        TR              TR_4     8000  72000  6000   96000
5    TR_5   TR     8        TR              TR_5     7000  63000  5250   84000
6    TR_6   TR     8        TR              TR_6     8000  72000  6000   96000
7    TR_7   TR    16        TR              TR_7    10000  90000  7500  120000
8    TR_8   TR     8        TR              TR_8     6000  54000  4500   72000
9    TR_9   TR    16        TR              TR_9     3000  27000  2250   36000
10  TR_10   TR     8        TR             TR_10     8000  72000  6000   96000
11  TR_11   TR    16        TR             TR_11    

/home/dlalvareza/anaconda3/lib/python3.7/site-packages/pandas/io/excel/_base.py:1292: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  **kwds,


,Name,Type,MTTR,Disc_Type,Asset_To_Disconet,Incomes,CAPEX,OPEX,Cr_Fin,Cr_Env,Cr_Sec,Cr_Leg,T_Cr
ID,,,,,,,,,,,,,
1,TR_1,TR,8,TR,TR_1,7000,63000,5250,84000,0.0,100000.0,0.0,184000.0
2,TR_2,TR,8,TR,TR_2,10000,90000,7500,120000,0.0,0.0,0.0,120000.0
3,TR_3,TR,8,TR,TR_3,10000,90000,7500,120000,1000.0,0.0,0.0,121000.0
4,TR_4,TR,8,TR,TR_4,8000,72000,6000,96000,1000.0,0.0,0.0,97000.0
5,TR_5,TR,8,TR,TR_5,7000,63000,5250,84000,0.0,0.0,0.0,84000.0
6,TR_6,TR,8,TR,TR_6,8000,72000,6000,96000,0.0,0.0,0.0,96000.0
7,TR_7,TR,16,TR,TR_7,10000,90000,7500,120000,0.0,0.0,0.0,120000.0
8,TR_8,TR,8,TR,TR_8,6000,54000,4500,72000,0.0,0.0,0.0,72000.0
9,TR_9,TR,16,TR,TR_9,3000,27000,2250,36000,0.0,0.0,0.0,36000.0


In [13]:
from PywerAM_Scenario_Assessment import Decision_Making

In [14]:
%%time
DMS            = Decision_Making(assets,DF_ACP=df_ACP,df_AC_Fixed=df_Fixed_Cr)
DMS.R          = 0.13   # Discount rate
DMS.date_beg   = date_beg
DMS.N_days     = d_day_for
# Scenario do nothing
DMS.run_scenario_base()

APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: overflow encountered in double_scalars
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: overflow encountered in double_scalars
  return (1 - np.exp(-((t*beta)**(1/alpha))))
APM/BIN/APM_Module_Tools.py:35: RuntimeWarning: overflow encountered in double_scalars
  return (1 - np.exp(-((t*beta)**(1/alpha))))


CPU times: user 2min 2s, sys: 17.1 s, total: 2min 19s
Wall time: 2min 20s


In [15]:
# Load Scenario do nothing
DMS.load_scenario_base()


\begin{equation*}
POF \left(  {\color{DColor9}t}\right)= 1-R\left( {\color{DColor9}t}\right)
\end{equation*}

\begin{equation*}
R \left(  {\color{DColor9}t} \right)= e^{-\int_{T_i}^{T_{end}}\lambda \left( {\color{DColor9}t}\right) \, dt}
\end{equation*}




\begin{equation*}
\lambda\left( {\color{DColor9}t}\right)= {\color{DColor3}a} \cdot e^{ {\color{DColor3}b}\cdot HI \left( {\color{DColor9}t}\right)}+ {\color{DColor3}c}
\end{equation*}

In [16]:
def update_HI_plot(Asset_Id):
    asset         = assets.Asset_Portfolio[Asset_Id]
    df            = DMS.scenario['Base'][Asset_Id]['Con'] #asset.POF_R_Assessment(date_beg,d_day_for*24)
    print(df.head())
    p    = plot_condition_assessment(df)
    show(p, notebook_handle=True)
    push_notebook()
    
interact(update_HI_plot, Asset_Id=asset_list_id)

interactive(children=(Dropdown(description='Asset_Id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=…

<function __main__.update_HI_plot(Asset_Id)>

# 3 Plot criticality

In [17]:
from bokeh.plotting import figure
def update_Cr_plot(Asset_Id):
    asset_name     = assets.Asset_Portfolio_List.loc[Asset_Id].Name
    df             = df_ACP[df_ACP[asset_name]==True]
    p              = figure(title="PyweAM - Asset: "+asset_name, plot_height=500, plot_width=950,background_fill_color='#808080',x_axis_type='datetime')
    p.circle(df["Date"], df["Cr"], fill_alpha=0.2, size=10)
    show(p, notebook_handle=True)
    push_notebook()
    
interact(update_Cr_plot, Asset_Id=asset_list_id)

interactive(children=(Dropdown(description='Asset_Id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=…

<function __main__.update_Cr_plot(Asset_Id)>

In [18]:
DMS.scenario['Base'][1]['RI']

,date,Cr,pof,RI,Inves
0,2022-12-31,-184000.000000,0.012781,-2351.651834,0
1,2023-12-31,-184000.000000,0.041830,-7696.637856,0
2,2024-12-31,-184000.000000,0.074701,-13745.009250,0
3,2025-12-31,-187637.661983,0.108348,-20330.210368,0
4,2026-12-31,-189062.945369,0.141954,-26838.243623,0
5,2027-12-31,-189715.359252,0.175177,-33233.821344,0
6,2028-12-31,-194220.336497,0.207934,-40385.035835,0
7,2029-12-31,-199695.220511,0.240198,-47966.407538,0
8,2030-12-31,-211182.466468,0.272070,-57456.481476,0
9,2031-12-31,-218152.167733,0.303952,-66307.683544,0


# 4 Risk Index
\begin{equation*}
RI_i \left( t \right) = POF_i \left( t \right) \times  Cr_i \left( t \right)
\end{equation*}

In [19]:
def update_Cr_plot(Asset_Id,Scenario):
    # # # # # # # # # #            
    asset_name     = assets.Asset_Portfolio_List.loc[Asset_Id].Name
    df              = DMS.scenario[Scenario][Asset_Id]['RI']
    p             = figure(title="PyweAM - Asset: "+asset_name, plot_height=500, plot_width=950,background_fill_color='#808080',x_axis_type='datetime')
    p.line(df["date"], -df["Cr"])
    p.line(df["date"], -df["RI"], color='#FF0000',)
    
    show(p, notebook_handle=True)
    push_notebook()
    print(df)

interact(update_Cr_plot, Asset_Id=asset_list_id,Scenario=DMS.scenario.keys())

interactive(children=(Dropdown(description='Asset_Id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=…

<function __main__.update_Cr_plot(Asset_Id, Scenario)>

## 5. Decision-Making
### Replacement strategies
#### Strategy 1

In [20]:
decision = {'Poject_ID': ['01','02','03','04'],
            'Asset_id': [1,3,6,7],
            'Des_Type': ['Replace','Replace','Replace','Replace'],
            'Cost'    : [2e4,2e4,2e4,2.25e4],
            'Impr'    : [1,1,1,1],
            'Date'    : [datetime.date(2028, 6, 1),datetime.date(2025, 6, 1),datetime.date(2027, 6, 1),datetime.date(2035, 6, 1)]
           }

df_dec         = pd.DataFrame.from_dict(decision)
s_name         = 'Scenario 1' 
DMS.run_scenario(s_name,df_dec)

UnboundLocalError: local variable 'name' referenced before assignment

#### Strategy 2

In [ ]:
decision = {'Poject_ID': ['01','02','03','04','05','06','07','08','09','10','11','12'],
            'Asset_id': [1,2,3,4,5,6,7,8,9,10,11,12],
            'Des_Type': ['Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace'],
            'Cost'    : [2e4,2e4,2e4,2.25e4,2e4,2e4,2e4,2.25e4,2e4,2e4,2e4,2.25e4],
            'Impr'    : [1,1,1,1,1,1,1,1,1,1,1,1],
            'Date'    : [datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),
                         datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),
                         datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1)]
           }
df_dec         = pd.DataFrame.from_dict(decision)
s_name         = 'Scenario_2' 
DMS.run_scenario(s_name,df_dec)

In [ ]:
def HI_plot(Asset_Id,Scenario):
    df = DMS.scenario[Scenario][Asset_Id]['Con']
    print(df.head())
    p    = plot_condition_assessment(df)
    show(p, notebook_handle=True)
    push_notebook()
    
interact(HI_plot, Asset_Id=asset_list_id,Scenario=DMS.scenario.keys())

### Financial assessment

In [ ]:
def PV_plot(Asset_Id,Scenario):
    df           = DMS.scenario[Scenario][Asset_Id]['CF']
    asset_name  = assets.Asset_Portfolio_List.loc[Asset_Id].Name
    print(df.head())
    p             = figure(title="PyweAM - Asset: "+asset_name, plot_height=500, plot_width=950,x_axis_type='datetime')
    p.vbar(x= df["date"], top=df["RI"], width=1e10)
    p.line(df["date"], df["Cash_flow"], color='#FF0000',)
    
    show(p, notebook_handle=True)
    push_notebook()

interact(PV_plot, Asset_Id=asset_list_id,Scenario=DMS.scenario.keys())

In [ ]:
def PV_by_scenario(dms):
    l_scen     = []
    l_asset_id = []
    l_asset_pv = []

    for sce in dms.scenario:
        l_scen_dic = dms.scenario[sce]
        for asset_id in l_scen_dic:  
            l_asset_dic = l_scen_dic[asset_id]
        
            l_scen.append(sce)
            l_asset_id.append(asset_id)
            l_asset_pv.append(l_asset_dic['PV'])

    l_dic = {'scenario': l_scen,
         'id': l_asset_id,
         'PV': l_asset_pv
        }        
    l_df = pd.DataFrame(l_dic)
    l_group = l_df.groupby(['scenario'])
    print(l_group['PV'].sum())
    return l_df

l_df = PV_by_scenario(DMS)    

In [ ]:
def PV_plot_DM(per,df):         
    df    = df.sort_values(by=['PV'])
    df    = df.head(int(len(df)*(per/100)))
    df.id = df.id.apply(str)
    p     = plot_decision_making(df)
    show(p, notebook_handle=True)
    push_notebook()
    
interact(PV_plot_DM, per=(0.0,100,5),df=fixed(l_df))

In [ ]:
def Risk_Forecast(scenario_name,plot,sce):
    p = plot_scenario(scenario_name,plot,sce)
    show(p, notebook_handle=True)
    push_notebook()

interact(Risk_Forecast, scenario_name=DMS.scenario.keys(),plot=['RI','Cum_Sum_PV'],sce=fixed(DMS.scenario))

#### Eval present value

In [ ]:
_=PV_by_scenario(DMS)  

In [ ]:
decision = {'Poject_ID': ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15'],
            'Asset_id': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
            'Des_Type': ['Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace','Replace'],
            'Cost'    : [2e4,2.25e4,2.25e4,2.25e4,2.25e4,2.25e4,2e4,2e4,2.25e4,2.25e4,2e4,2.25e4,2.25e4,2.25e4,2.25e4],
            'Impr'    : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            'Date'    : [datetime.date(2021, 6, 1),datetime.date(2021, 6, 1),datetime.date(2022, 6, 1),datetime.date(2022, 6, 1),
                         datetime.date(2023, 6, 1),datetime.date(2027, 6, 1),datetime.date(2023, 6, 1),datetime.date(2024, 6, 1),
                         datetime.date(2024, 6, 1),datetime.date(2029, 6, 1),datetime.date(2025, 6, 1),datetime.date(2031, 6, 1),
                         datetime.date(2031, 6, 1),datetime.date(2030, 6, 1),datetime.date(2030, 6, 1)]
                    }
df_dec         = pd.DataFrame.from_dict(decision)
s_name         = 'Guillermo_Yeison' 
DMS.run_scenario(s_name,df_dec)
